In [ ]:
# ==============================
# 1. Setup Environment
# ==============================
!pip install -q transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.5 MB/s eta 0:00:00


In [ ]:
!pip install huggingface_hub

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `token1` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential a

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

In [ ]:
# ==============================
# 2. Load Your Dataset
# ==============================
# Upload your qa_results_alpaca.jsonl to Colab first (Files panel -> Upload)
INPUT_FILE_NAME = '/content/aplaca_op.jsonl'
dataset = load_dataset("json", data_files=INPUT_FILE_NAME)
dataset = dataset["train"]   # select the split

print(dataset)  # sanity check

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 136
})


In [ ]:
# ==============================
# 3. Choose Base Model
# ==============================
# Pick a small model if GPU memory is limited (like 1-2B)
# Recommended options for Colab T4 / A100:
# "meta-llama/Llama-2-7b-hf"  (needs A100 or better)
# "NousResearch/Llama-2-7b-hf" (same as above)
# "mistralai/Mistral-7B-v0.1"  (lighter)
# "Qwen/Qwen1.5-1.8B"          (fits easily on Colab T4)
# We'll default to a smaller one here:

model_name = "Qwen/Qwen1.5-1.8B"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    low_cpu_mem_usage=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [ ]:
# ==============================
# 4. Prepare LoRA Config
# ==============================
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # for LLaMA/Qwen-style models
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
# ==============================
# 5. Tokenization Function
# ==============================
# Format prompt (like Alpaca)
# Format prompt
def format_prompt(example):
    if example["input"]:
        prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n"
    return prompt

# Tokenize
def tokenize(example):
    prompt = format_prompt(example)
    full_text = prompt + example["output"]

    tokenized = tokenizer(full_text, truncation=True, padding="max_length", max_length=512)

    # Labels
    labels = tokenized["input_ids"].copy()
    prompt_len = len(tokenizer(prompt)["input_ids"])
    labels[:prompt_len] = [-100] * prompt_len  # ignore prompt tokens

    tokenized["labels"] = labels
    return tokenized


In [ ]:
# ==============================
# 6. Training Arguments
# ==============================
training_args = TrainingArguments(
    output_dir="./alpaca-ft",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10
)

In [ ]:
# ==============================
# 7. Trainer
# ==============================

tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

Map:   0%|          | 0/136 [00:00<?, ? examples/s]

Step,Training Loss
10,5.111300
20,0.897000
30,0.105500
40,0.094100
50,0.093200


TrainOutput(global_step=51, training_loss=1.2376007817539514, metrics={'train_runtime': 120.6136, 'train_samples_per_second': 3.383, 'train_steps_per_second': 0.423, 'total_flos': 1916173100777472.0, 'train_loss': 1.2376007817539514, 'epoch': 3.0})

In [ ]:
# ==============================
# 8. Save Model
# ==============================
model.save_pretrained("./alpaca-ft")
tokenizer.save_pretrained("./alpaca-ft")

('./alpaca-ft/tokenizer_config.json',
 './alpaca-ft/special_tokens_map.json',
 './alpaca-ft/chat_template.jinja',
 './alpaca-ft/vocab.json',
 './alpaca-ft/merges.txt',
 './alpaca-ft/added_tokens.json',
 './alpaca-ft/tokenizer.json')

In [ ]:
# ==============================
# 9. Inference Test
# ==============================
from transformers import pipeline
prompt = """### Instruction:
What does the essay define consciousness as?

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


### Instruction:
What does the essay define consciousness as?

### Response:
The essay defines consciousness as the ability to think and act.
